# EDA and Preprocessing, CER

Author: Scott Yip \
Date: 28 June 2022

**PLEASE NOTE THAT THIS IS AN UPDATE OF 1.1 AS IT WAS DISCOVERED THAT 1.1 DID NOT PRODUCE OPTIMAL RESULTS DUE TO THE METHOD OF PREPROCESSING FOR THE DAILY NORMALIZATION.**

The abstract and information for this dataset can be accessed [here](https://www.ucd.ie/issda/data/commissionforenergyregulationcer/). In particular, we use Study Number (SN): 0012-00 - this has become a staple dataset for EED compression.

Additional information about the dataset is located in the manifest file labelled `Manifest - Smart Meter Electricity Trial Data v1.0.1.docx`.

The relevant smart meter read data is that

```
6 zipped files named File1.txt.zip to File6.txt.zip each containing 1 text file
Format of each data file: 3 columns corresponding to  
		Meter ID
		Five digit code:  
			Day code: digits 1-3 (day 1 = 1st January 2009)
			Time code: digits 4-5 (1-48 for each 30 minutes with 1= 00:00:00 – 00:29:59)
		Electricity consumed during 30 minute interval (in kWh)

```

The notable minimal preprocessing changes we'll have to make are:
* conversion of five digit code to timedate format.

We further note that we work with $30$ minute intervals.

## 1. Intro and baseline processing

### 1A. Intro

Let's perform some very quick EDA to get an idea of this dataset.

Nb: we use the `dask` library instead of `pandas` as we want to avoid out-of-memory computations. `dask` will allow us to perform on-disk processing for larger-than-memory computations (which may occur).

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import os
import dask.dataframe as dd
import tqdm
import random

In [2]:
# Set folder parameters
dataset_folder = "../../dataset/raw/CERN/"
output_folder = "../../dataset/interim/"

In [3]:
pd.set_option('display.max_rows', 500)
%matplotlib inline

As the files are all separate, we'd like to use `dask` to read all the files.

Let's use `os.listdir()` to check all files (non recursively).

In [4]:
# use os.listdir() and not os.walk() as no recursive movement required
all_files = [dataset_folder + file for file in os.listdir(dataset_folder) if file[-3:] == 'txt']

print('Number of txt files: {}'.format(len(all_files)))

Number of txt files: 6


We use `pandas` directly this time as we can.

In [5]:
df = pd.DataFrame()

for f in tqdm.tqdm(all_files):
    
    temp_ = pd.read_csv(f, names=['metre_id', 'timecode', 'kwh'], sep=' ',
                dtype={'metre_id': 'int64', 'timecode': 'object', 'kwh': 'float64'})
    
    df = pd.concat([df, temp_])

100%|██████████| 6/6 [00:42<00:00,  7.03s/it]


In [6]:
df.head(3)

,metre_id,timecode,kwh
0,4597,19711,0.144
1,4597,19712,0.144
2,4597,19501,0.208


Timecode conversion:

In [7]:
def time_conversion(df, timecode='timecode'):
    
    """
    Auxiliary function to convert timecodes to date codes
    """
    
    # Implement the intial date (1/1/19)
    init_date = datetime.datetime(2009, 1, 1)
    
    # Split out the timecodes to day codes and minute codes
    day_code = df[timecode].str.slice(stop=3).astype('int64')
    minute_code = df[timecode].str.slice(start=3).astype('int64')
    
    # Note: dd.to_timedelta() does not seem to work with dd.map_partitions()
    # resorting to using pd.to_timedelta() though I believe this is slower
    
    # Add day code
    temp_date = init_date + pd.to_timedelta(day_code-1, unit='day')
    # Add minute code
    temp_date = temp_date + pd.to_timedelta(minute_code*30, unit='minute')
    # Reduce by one second to keep within same day
    temp_date = temp_date - pd.to_timedelta(1, unit='seconds')
    
    return temp_date

In [8]:
# Create timestamp column
df['timestamp'] = time_conversion(df)

In [9]:
df_memory = df

In [10]:
len(df)

157992996

In [11]:
len(df_memory)

157992996

#### Save Start

In [12]:
df_memory = df

#### Average duplicates:

In [13]:
df_memory = df_memory.groupby(['metre_id', 'timestamp']).mean().reset_index()

In [14]:
len(df_memory)

157967988

#### Make Test

In [15]:
df_mini = df_memory[:500000]

#### Resample and find missing.

In [16]:
df_memory = df_memory.set_index('timestamp').groupby('metre_id').resample('30T', origin='start').ffill(limit=5)

#### Check NA per Meter

In [17]:
check_na = df_memory.metre_id.isna().groupby(level=0).sum()

In [18]:
no_nils = check_na[check_na == 0].index

In [19]:
no_nils

Int64Index([1000, 1001, 1002, 1003, 1004, 1005, 1006, 1009, 1010, 1011,
            ...
            7433, 7434, 7435, 7436, 7437, 7438, 7441, 7442, 7443, 7444],
           dtype='int64', name='metre_id', length=5202)

#### Remove

Only keep full records per meter.

In [20]:
df_memory = df_memory[df_memory.index.isin(no_nils, level = 0)]

In [21]:
len(df_memory)

127161962

#### Cut off at Midnight

In [22]:
# Push timestamp back into columns
df_memory = df_memory.reset_index(level = 1)

In [23]:
def cut_off_midnights(subset_df):

    if subset_df.timestamp.min().time() != datetime.time(0, 29, 59):
        first_day = subset_df.timestamp.min().date()
        filter_date_start = pd.to_datetime(first_day) + pd.to_timedelta('1 day') + pd.to_timedelta('0 hours 29 minutes 59 seconds')
        subset_df = subset_df[subset_df['timestamp'] >= filter_date_start]
        
    if subset_df.timestamp.max().time() != datetime.time(23, 59, 59):
        last_day = subset_df.timestamp.max().date()
        filter_date_last = pd.to_datetime(last_day) - pd.to_timedelta('1 day') + pd.to_timedelta('23 hours 59 minutes 59 seconds')
        subset_df = subset_df[subset_df['timestamp'] <= filter_date_last]
        
    return subset_df.reset_index(drop=True)

In [24]:
df_memory = df_memory.groupby(level = 0).apply(cut_off_midnights).reset_index(drop=True, level = 1)

In [25]:
len(df_memory)

127161936

#### Average Again

In [26]:
df_memory = df_memory.reset_index(drop = True)

In [27]:
# Average duplicates again
df_memory = df_memory.groupby(['metre_id', 'timestamp']).mean().reset_index()

In [28]:
len(df_memory)

127161936

#### Find Which Day is the most Popular

In [29]:
df_memory['date'] = df_memory.timestamp.dt.date

In [30]:
check_day_popularity = df_memory.drop_duplicates(['metre_id', 'date']).groupby('date').kwh.count()

In [31]:
check_day_popularity[:50]

date
2009-07-14    5169
2009-07-15    5171
2009-07-16    5172
2009-07-17    5171
2009-07-18    5171
2009-07-19    5171
2009-07-20    5170
2009-07-21    5169
2009-07-22    5167
2009-07-23    5165
2009-07-24    5163
2009-07-25    5164
2009-07-26    5164
2009-07-27    5164
2009-07-28    5160
2009-07-29    5160
2009-07-30    5159
2009-07-31    5157
2009-08-01    5157
2009-08-02    5157
2009-08-03    5158
2009-08-04    5158
2009-08-05    5157
2009-08-06    5156
2009-08-07    5155
2009-08-08    5155
2009-08-09    5155
2009-08-10    5151
2009-08-11    5151
2009-08-12    5149
2009-08-13    5148
2009-08-14    5147
2009-08-15    5147
2009-08-16    5148
2009-08-17    5148
2009-08-18    5146
2009-08-19    5145
2009-08-20    5142
2009-08-21    5141
2009-08-22    5141
2009-08-23    5141
2009-08-24    5140
2009-08-25    5139
2009-08-26    5135
2009-08-27    5135
2009-08-28    5134
2009-08-29    5134
2009-08-30    5134
2009-08-31    5133
2009-09-01    5132
Name: kwh, dtype: int64

Check for all zeroes OR all same (errors I ran into)

In [32]:
def all_zero_or_same_checker(subset_df):

    # return all(subset_df.kwh == 0) or (len(subset_df.kwh.unique()) <= 1)

    all_zero_ = all(subset_df.kwh == 0)
    uniques_ = len(subset_df.kwh.unique())

    temp_ = pd.Series([all_zero_, uniques_])

    return temp_ 

In [33]:
all_zero_checker = df_memory.groupby(['metre_id', 'date']).sum()

In [34]:
all_unique_checker = df_memory.groupby(['metre_id', 'date']).kwh.nunique()

In [35]:
both_checker = all_zero_checker.reset_index()[(all_unique_checker.reset_index().kwh <= 1) |\
                               (all_zero_checker.reset_index().kwh == 0)][['metre_id', 'date']].itertuples(index=False, name=None)

In [36]:
zero_indexer = df_memory.set_index(['metre_id', 'date']).index.isin(both_checker)

In [37]:
df_memory = df_memory[~zero_indexer]

In [38]:
len(df_memory)

126483072

Use 100 households for 35 days. Let's say from Sep 1 to Oct 30.

In [39]:
keep_date_range = pd.date_range(start = '2009-09-01', end = '2009-10-30', freq = '1D')

In [40]:
def check_all_dates_in_range(subset_df):
    
    return all(keep_date_range.isin(subset_df.date))

In [41]:
all_date_consec_checker = df_memory.groupby('metre_id').apply(check_all_dates_in_range)

In [42]:
all_date_consec_checker

metre_id
1000.0     True
1001.0     True
1002.0     True
1003.0     True
1004.0     True
          ...  
7438.0    False
7441.0     True
7442.0     True
7443.0     True
7444.0     True
Length: 5202, dtype: bool

In [43]:
sum(all_date_consec_checker)

5001

In [44]:
k = 5000
# houses_selected = np.random.choice(all_date_consec_checker[all_date_consec_checker == True].index, k, replace = False) # random
houses_selected = all_date_consec_checker[all_date_consec_checker == True].index[:k].values # first k

#### Pull Selected Houses

In [45]:
df_memory.head()

,metre_id,timestamp,kwh,date
0,1000.0,2009-07-14 00:29:59,0.117,2009-07-14
1,1000.0,2009-07-14 00:59:59,0.050,2009-07-14
2,1000.0,2009-07-14 01:29:59,0.025,2009-07-14
3,1000.0,2009-07-14 01:59:59,0.025,2009-07-14
4,1000.0,2009-07-14 02:29:59,0.049,2009-07-14


In [46]:
df_processed = df_memory[(df_memory.timestamp.dt.normalize().isin(keep_date_range)) & (df_memory.metre_id.isin(houses_selected))]

Okay, so here we expected the $\text{num houses} \times \text{date range} \times 48$ so:

In [47]:
len(keep_date_range)

60

In [48]:
k * len(keep_date_range) * 48

14400000

In [49]:
len(df_processed)

14400000

Great.

#### Normalize

**DO NOT USE** for UNNORMAL.
PREPROCESSING DONE IN DATASET LOADER.

In [50]:
# def normalize_daily_load_profiles(subset_df):
    
#     subset_df.kwh = (subset_df.kwh - subset_df.kwh.min()) / (subset_df.kwh.max() - subset_df.kwh.min())
    
#     return subset_df

In [51]:
# df_processed = df_processed.groupby(['metre_id', 'date']).apply(normalize_daily_load_profiles)

#### TT Split

In [52]:
train_k = int(k*.9)

### Random
# houses_train = np.random.choice(houses_selected, train_k, replace = False)
# houses_test = np.setdiff1d(houses_selected, houses_train) 

### Deterministic
houses_train = houses_selected[:train_k]
houses_test = houses_selected[train_k:]

In [53]:
df_final = df_processed.drop('date', axis = 1)

In [54]:
df_train = df_final[df_final.metre_id.isin(houses_train)]
df_test = df_final[df_final.metre_id.isin(houses_test)]

In [55]:
len(df_train)

12960000

In [56]:
len(df_test)

1440000

#### Save

In [251]:
# df_train.to_csv(output_folder + 'cern_train_large_unnormal.csv', index=False)
# df_test.to_csv(output_folder + 'cern_test_large_unnormal.csv', index=False)

***